# Making allele frequency plots

This notebook contains code to process the CRISPResso "allele frequency table" files from base editor validation experiments. The inputs are 2 input files: the first contains metainformation about each sample to make the "allele frequency" file, and the second contains metainformation to compute correlations between the log-normalized read counts. The output of this file are 3 files for each sgRNA / primer pair: 
1. a file containing all alleles and their read counts for each sample
2. a filtered version of (1) that only contains alleles with at least 1% abundance in any sample
3. a file containing the Pearson correlations between log-normalized read counts of each allele with > 100 reads in at least one sample

(1) is the starting file used to show the abundance of specific edits over time (code in BEV_aa_over_time.ipynb). (2) is the starting file used to create allele-level heatmaps (this was done using GraphPad Prism). (3) is the starting file for the plots showing the replicate correlation for validation experiments (actual plots were made using GraphPad Prism).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import HTML
import random

def hide_toggle(toggle_text_addon = '', for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'
    
    toggle_text = 'Show/Hide' + ' ' + toggle_text_addon  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)
hide_toggle(toggle_text_addon='Toggle Function')

In [4]:
import sys
sys.path.append('../scripts/')
import pandas as pd 
import numpy as np 
import core_functions as cfs
from math import log
from os import path
import itertools
import re
from pathlib import Path
from tqdm import tqdm

pd.set_option('display.max_colwidth', None)

In [5]:
print('Python version: ' + sys.version)

Python version: 3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


In [6]:
modules = ['pandas', 'numpy']
for module in modules:
    try:
        print(module + ' ' + sys.modules[module].__version__)
    except:
        print(module + ' has no __version__ attribute')

pandas 1.2.1
numpy 1.19.5


## Functions

In [7]:
'''
This function removes any NaN rows from input_file
'''
def clean_input_file(df):
    df = df.dropna() #drop NaN 
    #dropna() converts int to float, so convert them back
    float_cols = df.select_dtypes(include=['float64']).columns #select subset of df of type float
    for col in float_cols: 
        #get original column index so can replace at correct loc
        index = df.columns.get_loc(col)
        #rename float cols as "float_"col name 
        float_col_name = 'float_' + col
        df = df.rename(columns = {col : float_col_name})
        #overwrite as type int
        float_to_int = df[float_col_name].astype(int).copy()
        df.insert(index, col, float_to_int)
        #drop float column 
        df = df.drop(float_col_name, axis = 1)
    return df

hide_toggle(toggle_text_addon='clean_input_file function')

In [8]:
'''
Called from: process_data_v2
Function: removes UTRs from Aligned_Sequence before translation

INPUTS
------
translation_ref_seq: reference sequence from input file with intron sequence indicated by lowercase 
aligned: Aligned_sequence from merge df 
rev_com: True (from input file) if sequence needs to be reverse complemented 

OUTPUTS
-------
new_aligned: aligned sequence without UTRs and reverse complemented if necessary 

'''
def remove_utr(translation_ref_seq, aligned, rev_com):
    
    if ('[' not in translation_ref_seq) or (']' not in translation_ref_seq): # if no UTR
        return
    
    open_bracket = False # True when encounters open bracket [
    closing_bracket = False # True when encounters closed bracket ]
    in_utr = False # True when open_bracket == True and closing_bracket == False
    utr_list = []
    utr = ''
    utr_start_end_list = []
    start_end = [] 
    bracket_count = 0
    # check to make sure all open brackets are only followed by closed, and closed is never followed by closed
    for idx, char in enumerate(translation_ref_seq):
        if char == '[':
            if open_bracket: # if already encountered open bracket
                return 'Open bracket encountered after open bracket. Please make sure all brackets are closed.'
            else:
                open_bracket = True
                closing_bracket = False
                
        if char == ']':
            if open_bracket: # if this ] closes encountered open bracket
                open_bracket = False
                closing_bracket = True
                bracket_count +=1 # count closed bracket 
                in_utr = False # end of UTR
                utr_list.append(utr) # add UTR sequence to list
                start_end.append(idx-bracket_count) # subtract # brackets encountered
                utr_start_end_list.append(start_end)
                utr = '' # Reset UTR
                start_end = [] # Rest start_end index pair
                
            elif closing_bracket: # if already encountered closing bracket
                return 'Closing bracket encountered after closing bracket. Please check input.'
            else:
                return 'No open bracket encountered. Please check input.'
        
        if (open_bracket and not closing_bracket) and (char != '[') and (char != ']'):
            in_utr = True 
            # if first character in utr, store start index
            if len(utr) == 0:
                bracket_count +=1 # count open bracket
                start_end.append(idx-bracket_count) # subtract # brackets encountered
            utr = utr + char
    
#     print(utr_list, utr_start_end_list)
    
    new_aligned = aligned 
    new_utrs = []
    new_utr_start_end_list = []
    if rev_com: # need to reverse complement UTR from translation_ref_seq to match aligned seq
        # Reverse UTR index start, end positions if rev_com = True
        for i in utr_start_end_list:
            new_idx_start = -(i[1]+1) # reverse index, +1 b/c starts at -1 in reverse
            # convert to positive index
            new_idx_start = len(translation_ref_seq) + new_idx_start
            new_idx_end = -(i[0]+1) # reverse index, +1 b/c starts at -1 in reverse
            # convert to positive index
            new_idx_end = len(translation_ref_seq) + new_idx_end
            new_utr_start_end_list.append([new_idx_start, new_idx_end])
#       
        for utr in utr_list:
            rev_com_utr = cfs.revcom(utr)
            new_utrs.append(rev_com_utr)
    else:
        new_utr_start_end_list = utr_start_end_list
        new_utrs = utr_list
    
#     print(utr_start_end_list, new_utr_start_end_list)
#     print(utr_list, new_utrs)
    len_prev_utr = 0
    for idx, utr in enumerate(new_utrs):
        # if not first UTR, adjust start and end index based on length of prev UTR
        start_end_idx = new_utr_start_end_list[idx]
        if start_end_idx[0] < 0: # if negative index, add len_prev_utr
            start_end_idx = [start_end_idx[0]+len_prev_utr, start_end_idx[1]+len_prev_utr]
        else:
            start_end_idx = [start_end_idx[0]-len_prev_utr, start_end_idx[1]-len_prev_utr]
#         print(idx, start_end_idx, utr)
        # Check if bases at intron positions in aligned seq match utr
        new_aligned_intron = new_aligned[start_end_idx[0]:start_end_idx[1]+1]
        if new_aligned_intron != utr.upper():
#             print(new_aligned_intron)
#             print('not found')
            new_aligned = 'UTR'
            return new_aligned
        else:
#             print('found')
            # remove intron based on index position 
            if start_end_idx[0] == 0: # if intron at the beginning of new_aligned seq, keep substring after end_index
                new_aligned = new_aligned[start_end_idx[1]+1:]
            else: # (bases until start index + bases after end index)
                new_aligned = new_aligned[:start_end_idx[0]]+new_aligned[start_end_idx[1]+2:]
#             print(new_aligned)
            len_prev_utr = len(utr)+1 # adjust for zero index
        
    if rev_com: # Now that UTR has been found, reverse complement again so correct translation input
        new_aligned = cfs.revcom(new_aligned)
# #     print(new_aligned)
    return new_aligned

hide_toggle(toggle_text_addon='remove_utr function')

In [9]:
from itertools import groupby
from operator import itemgetter
'''
Called from: process_data_v2
Function: removes introns from Aligned_Sequence before translation

INPUTS
------
translation_ref_seq: reference sequence from input file with intron sequence indicated by lowercase 
aligned: Aligned_sequence from merge df 
rev_com: True (from input file) if sequence needs to be reverse complemented 

OUTPUTS
-------
new_aligned: aligned sequence without introns and reverse complemented if necessary 

'''
def remove_introns(translation_ref_seq, aligned, rev_com):
    # get index positions of introns 
    intron_idx= [i for i, base in enumerate(translation_ref_seq) if base.islower()]
    intron_idx_list = []
    # Get consecutive indices to group indices for same intron
    for k, g in groupby(enumerate(intron_idx), lambda ix : ix[0] - ix[1]):
         intron_idx_list.append(list(map(itemgetter(1), g)))
    
    # if no introns, exit function
    if not intron_idx_list:
        #print('no introns')
        return 
    
    # Get sequence of intron to check for intronic mutations 
    introns = []
    intron_start_end_list = []
    for intron_idx_group in intron_idx_list:
        intron_start = intron_idx_group[0]
        intron_end = intron_idx_group[-1]
        # store intron start and end index positions in pairs in list 
        intron_start_end_list.append([intron_idx_group[0], intron_idx_group[-1]])
#         print(intron_start_end_list)
        intron = translation_ref_seq[intron_start:intron_end+1]
        introns.append(intron)
        print(introns)
    
    new_aligned = aligned.upper()
    
    new_intron_start_end_list = []
    new_introns = []    

    if rev_com:
        # Reverse intron index positions if rev_com = True
        for i in intron_start_end_list:
            new_idx_start = -(i[1]+1) # reverse index
            # convert to positive index
            new_idx_start = len(translation_ref_seq) + new_idx_start
            new_idx_end = -(i[0]+1) # reverse index
            # convert to positive index
            new_idx_end = len(translation_ref_seq) + new_idx_end
            new_intron_start_end_list.append([new_idx_start, new_idx_end])
#             print(i, new_intron_idx_list)
        for i in introns:
            rev_com_intron = cfs.revcom(i.upper())
            new_introns.append(rev_com_intron)
#             print(new_introns)
    else:
        new_intron_start_end_list = intron_start_end_list
        new_introns = introns
    
    print(new_intron_start_end_list)
    print(new_introns, new_aligned)
    len_prev_intron = 0
    for idx, intron in enumerate(new_introns):
        # if not first intron, adjust start and end index based on length of prev intron
        start_end_idx = new_intron_start_end_list[idx]
        if start_end_idx[0] < 0: # if negative index, add len_prev_utr
            start_end_idx = [start_end_idx[0]+len_prev_intron, start_end_idx[1]+len_prev_intron]
        else:
            start_end_idx = [start_end_idx[0]-len_prev_intron, start_end_idx[1]-len_prev_intron]
#         print(idx, start_end_idx, intron)
        # Check if bases at intron positions in aligned seq match intron
        if new_aligned[start_end_idx[0]:start_end_idx[1]+1] != intron.upper():
#             print('not found')
            new_aligned = 'intron'
            return new_aligned
        else:
#             print('found')
            # remove intron based on index position 
            if start_end_idx[0] == 0: # if intron at the beginning of new_aligned seq, keep substring after end_index
                new_aligned = new_aligned[start_end_idx[1]+1:]
#                 print(new_aligned)
            else: # (bases until intron start index + bases after intron end index)
                new_aligned = new_aligned[:start_end_idx[0]]+new_aligned[start_end_idx[1]+1:]
#             print(new_aligned)
            len_prev_intron = len(intron)+1 # adjust for zero index
#     print(new_aligned)
    if rev_com:
        new_aligned = cfs.revcom(new_aligned)
#     print('introns removed, new aligned:', new_aligned)
    return new_aligned

hide_toggle(toggle_text_addon='remove_introns function')

In [10]:
# This function returns the translation of a given sequence and frame
def translate(seq, frame, first_codon, last_codon, codon_map):
#     if not seq: # if remove_introns returned False -> possible splice site mutation
    if seq == 'UTR':
        return 'Possible UTR mutation'
    if seq == 'intron':
        return 'Possible intronic mutation'
    aa = ''
    i = 0
    # if frame is not 1, add nucleotides necessary to beginning of sequence for translation based on first_codon
    if frame == 2:
        seq = first_codon[0] + seq
        frame = 1
    if frame == 3:
        seq = first_codon[0:2] + seq
        frame = 1

#     print(seq)    
    while i < len(seq):
        substring = ''
        while frame <= 3:
            if i<len(seq):
                if seq[i] == '-':
                    #print('deletion')
                    i += 1
                    # frame doesn't change
                else:
                    substring += seq[i]  
                    i += 1
                    frame+=1
            else: # if reached end of the sequence and frame still <=3, complete codon sequence based on last_codon
                substring += last_codon[frame-1]
                i += 1
                frame+=1
    
#             print('substring:', substring, 'i = ', i, 'frame = ', frame)
        if len(substring) == 3:
            frame = 1 # reset frame 
            if ('N' in substring):
                aa = aa + '-'
            else:
                aa = aa + codon_map[substring] # translate codon
#             print(aa)
        else:
            frame = 1
#         print(frame)
    return aa

codon_map = {'TTT':'F', 'TTC':'F', 'TTA':'L', 'TTG':'L', 'CTT':'L', 'CTC':'L', 'CTA':'L', 'CTG':'L', 'ATT':'I', 'ATC':'I',
             'ATA':'I', 'ATG':'M', 'GTT':'V', 'GTC':'V', 'GTA':'V', 'GTG':'V', 'TCT':'S', 'TCC':'S', 'TCA':'S', 'TCG':'S',
             'CCT':'P', 'CCC':'P', 'CCA':'P', 'CCG':'P', 'ACT':'T', 'ACC':'T', 'ACA':'T', 'ACG':'T', 'GCT':'A', 'GCC':'A',
             'GCA':'A', 'GCG':'A', 'TAT':'Y', 'TAC':'Y', 'TAA':'*', 'TAG':'*', 'CAT':'H', 'CAC':'H', 'CAA':'Q', 'CAG':'Q',
             'AAT':'N', 'AAC':'N', 'AAA':'K', 'AAG':'K', 'GAT':'D', 'GAC':'D', 'GAA':'E', 'GAG':'E', 'TGT':'C', 'TGC':'C',
             'TGA':'*', 'TGG':'W', 'CGT':'R', 'CGC':'R', 'CGA':'R', 'CGG':'R', 'AGT':'S', 'AGC':'S', 'AGA':'R', 'AGG':'R',
             'GGT':'G', 'GGC':'G', 'GGA':'G', 'GGG':'G'}

hide_toggle(toggle_text_addon='translate function')

In [11]:
def check_input_file(input_file):
    
    print('Checking input file...')
    # Check each input column for correct formatting

    err_flag = False

    for i, row in tqdm(input_file.iterrows()):
        translation_err_flag = False
        utr_flag = False
        intron_flag = False
        sg = row['sg']
        print('sg:', sg)
        # sgRNA_sequence must be all caps
        input_file.loc[i, 'sgRNA_sequence'] = row['sgRNA_sequence'].upper()
        # translation_ref_seq: use WT aligned condition to check formatting, frame, first codon, last codon
        ref_seq = row['translation_ref_seq']
        print('Translation reference sequence:', ref_seq)
        WT_aligned_seq = ref_seq
        brackets = '[]'
        # if UTR in translation_ref_seq
        if ('[' in ref_seq) or (']' in ref_seq):
            utr_flag = True
            for bracket in brackets:
                WT_aligned_seq = WT_aligned_seq.replace(bracket, '')
        if any(c.islower() for c in ref_seq):
            intron_flag = True
            WT_aligned_seq = WT_aligned_seq.upper()
        
        
        translation_input = WT_aligned_seq

        # Remove UTRs and/or introns if applicable to check frame, first codon, and last codon 
        # Since translation_ref_seq in correct orientation for translation, rev_com = False for this check
        rev_com = False
        if utr_flag:
            translation_input = remove_utr(ref_seq, translation_input, rev_com)
        if intron_flag:
#             print(ref_seq, translation_input, rev_com)
            translation_input = remove_introns(ref_seq, translation_input, rev_com)
        
        # Match frame and translation_input with first codon to check if they match
        frame = row['frame']
        first_codon = row['first_codon']
        last_codon = row['last_codon']
        translation_start_idx = frame - 1
        print('Translation input:', translation_input)
        # Check if first base in translation_input matches corresponding nucleotide in first codon based on frame 
    #     print(frame, translation_input[0], first_codon[translation_start_idx])
        if translation_input[0] != first_codon[translation_start_idx]:
            print(translation_input[0], first_codon[translation_start_idx])
            print('Frame does not match base in first_codon. Please check inputs') 
            err_flag = True
#             translation_err_flag = 
        else:
            # Check if last codon in translation_ref_seq is complete
            # length of translation_input - # bases in start codon present in translation_ref_seq
            print(len(translation_input), translation_start_idx)
            len_last_codon = (len(translation_input) - (3-translation_start_idx))%3 
#             len_last_codon = (len(translation_input) - frame)%3 
            print(len_last_codon)
    #         print(len_last_codon, translation_input[-len_last_codon:])
            if len_last_codon == 0:
                if translation_input[-3:] != last_codon:
                    print(translation_input[-3:], last_codon)
                    print('Input for last_codon does not align with translation_ref_seq. Please check inputs.')
                    err_flag = True
            else:
#                 print(translation_input, translation_input[-len_last_codon:], last_codon)
                if translation_input[-len_last_codon:] != last_codon[:len_last_codon]:
                    print(translation_input[-len_last_codon:], last_codon[:len_last_codon])
                    print('Input for last_codon does not align with frame and translation_ref_seq. Please check inputs.')
                    err_flag = True

            if not err_flag:
                print('Expected WT Translation: ', translate(translation_input, frame, first_codon, last_codon, codon_map))
#         break
    if not err_flag:
        return('Input file is correct!')
    else:
        return('Please address errors listed above before proceeding.')

hide_toggle(toggle_text_addon='check_input_file function')

In [12]:
'''
Called from: process_data_v2
Function: converts the BEV number from an int to a 3-digit string
'''
def get_bev_str(bev):
    bev = int(bev)
    if bev < 10:
        return '00'+str(bev)
    if bev < 100:
        return '0'+str(bev)
    return str(bev)
'''
Called from: get_path
Function: Verifies existence of filepath generated by get_path function to retrieve 'Alleles_frequency_table_around_sgRNA' file

INPUTS
------
filepath : filepath to 'Alleles_frequency_table_around_sgRNA_[].txt' file in CRISPResso output folder, 
           from get_path function
bev : BEV sample number 
sg_seq: guide sequence

OUTPUTS
-------
file_loc : filepath of 'Alleles_frequency_table_around_sgRNA_[].txt' file if exists
None if no filepath exists 

'''
def check_filepath(filepath,bev,primer,sg_seq):
    file_loc = filepath+'CRISPResso_on_'+bev+'_'+primer+'/'+'Alleles_frequency_table_around_sgRNA_'+sg_seq+'.txt'
    print(file_loc)
    if path.exists(file_loc):
        return file_loc
    else:
        raise ValueError('File not found. Please check your filepath inputs.')
        return ''

'''
Called from: get_bev_files
Function: calls check_filepath to get filepath where CRISPResso output files are stored, 
whose folder path (CRISPResso_filepath) is provided above
Calls: check_filepath

INPUTS
------
bev_num : BEV sample number
primer: primer name (from input file)
sg_seq: guide sequence

OUTPUTS
-------
file_loc : filepath of 'Alleles_frequency_table_around_sgRNA_[].txt' file if exists
None if no filepath exists 

'''

def get_path(bev_num,primer,sg_seq):
    bev = bev_string_id + '_' + get_bev_str(bev_num)
    filepath = CRISPResso_filepath
    return check_filepath(filepath,bev,primer,sg_seq)


'''
Called from: get_bev_files
Function: merges together the "Allele_frequency_table_around_sgRNA" files

INPUTS
------
filepath : filepath to 'Alleles_frequency_table_around_sgRNA_[].txt' file in CRISPResso output folder, 
           from get_path function
bev : BEV sample number 
sg_seq: guide sequence
existing_df: merge data frame with 'Aligned_Sequence' and 'Reference_Sequence' columns defined in get_bev_files
cols: empty list populated in function 

OUTPUTS
-------
merge : merged dataframe with Aligned_Sequence, Reference_Sequence, Reads columns from each sample 
cols: columns labeled with BEV sample number 
'''
def merge_bev_file(filepath,bev,sg_seq,existing_df,cols):
    if not path.exists(filepath):
        print('no file')
        return existing_df,cols
    df = pd.read_table(filepath,index_col=False)
    # Sum together any rows that share both 'Aligned Sequence' and 'Reference Sequence' with each other (this is rare)
    df_summed = df[['Aligned_Sequence','Reference_Sequence','#Reads','%Reads']].groupby(['Aligned_Sequence','Reference_Sequence'],as_index=False).agg('sum')
    cols.append(str('_BEV_'+str(bev)))
    df_summed = df_summed.rename(columns={'#Reads':str('#Reads_BEV_'+str(bev)),'%Reads':str('%Reads_BEV_'+str(bev))})
    # Outer merge onto existing dataframe
    merge = pd.merge(existing_df,df_summed,how='outer',on=['Aligned_Sequence','Reference_Sequence'])
    # Fill in nans with 0
    merge = merge.fillna(0)
    return merge,cols

'''
Called from: process_data_v2
Function: function gets and merges all "Allele_frequency_table_around_sgRNA" files for a given sgRNA sequence
(i.e. different replicates, drug conditions, etc.)
This is customized to work with files with the "BEV" notation
Calls: get_path, merge_bev_file

INPUTS
------
bev_list : contains BEV sample number, primer name, guide sequence for each sample  

OUTPUTS
-------
merge: merged dataframe with 
cols:

'''

def get_bev_files(bev_list):
    merge = pd.DataFrame(columns=['Aligned_Sequence','Reference_Sequence'])
    cols = []
    for bev,sg_seq,primer_name in bev_list:
        filepath = get_path(bev,primer_name,sg_seq)
        if filepath != '':
            merge,cols = merge_bev_file(filepath=filepath,bev=bev,sg_seq=sg_seq,existing_df=merge,cols=cols)
    return merge,cols

'''
Called from: process_data_v2
Function: filters out rows that don't meet given threshold (<1% for %Reads and <100 for #Reads)

INPUTS
------
row : row in column to which function is being applied (%Reads or #Reads)
cols: given cols (%Reads, #Reads for all samples )
val: threshold for filter 

OUTPUTS
-------
returns False for any rows (alleles) that have a value < the given value in ALL of the given cols

'''
def read_count_filter(row,cols,val):
    for col in cols:
        if row[col] > val:
            return True
    return False


'''
This function checks for WT allele (if Aligned_Sequence = Reference_Sequence)

INPUTS
------
row : row of merge dataframe 

OUTPUTS
-------
returns True if the allele is unedited (i.e. WT) and False otherwise

'''
def get_wt_col(row):
    if row['Aligned_Sequence'] == row['Reference_Sequence']:
        return True
    else:
        return False

'''
This function calculates the LFC

INPUTS
------
row : row of metainformation input file containing BEV_test and BEV_ref columns
data_file : merged dataframe containing log-normalized rpm for each allele

OUTPUTS
-------
data_file : merged dataframe, now with LFC columns for each BEV test / ref pair

'''
def get_lfc_v2(row,data_file):
    cols = []
    bev_list = row['BEV_test'].split(';')
    
    # Go through each test sample in BEV_test column
    for i,bev in enumerate(bev_list):
        test = get_bev_str(bev)
        
        # Get reference sample for LFC from BEV_ref column
        ref = get_bev_str(row['BEV_ref'].split(';')[i])
        
        # Calculate LFC
        data_file['LFC_'+test+'-'+ref] = data_file['#Reads_BEV_'+test+';lognorm'] - data_file['#Reads_BEV_'+ref+';lognorm']  
        cols.append('LFC_'+test+'-'+ref)
        
    # Average together LFC columns
    data_file['AvgLFC_'+'_'.join(bev_list)] = data_file.loc[:,cols].mean(axis=1)
    return data_file

'''
Called from: run
Function: merges read counts (filtered), lognorms, aligned sequences, reference sequences, translations from 
test and reference samples for sgRNA 

INPUTS
------
data : deduplicated input file with column 'sg','BEV_start','BEV_end','sgRNA_sequence','primer','frame','rev_com'

OUTPUTS
-------
merge : merged dataframe with filtered read counts, lognorms, aligned sequences, reference sequences, translations

'''
        
def process_data_v2(data):
    bev_list = [] # list to store info for retrieving CRISPResso files for given sgRNA
    ref_nums = [] # list to store BEV numbers for CRISPResso files for reference samples 
    for i,row in data.iterrows():
        for bev in range(row['BEV_start'],row['BEV_end']+1):
            # store BEV number, sgRNA sequence, and primer name to get corresponding CRISPResso files 
            bev_list.append((get_bev_str(bev),row['sgRNA_sequence'],row['primer']))
        
        # if reference samples outside of BEV_start and BEV_end range, add to bev_list separately
        BEV_ref = data['BEV_ref'].copy().tolist() #.copy.loc['BEV_ref']))
        BEV_ref_split = BEV_ref[0].split(';')
        for num in BEV_ref_split:
            # check if reference sample numbers outside of BEV_start and BEV_end range
            if ((int(num) < row['BEV_start']) | (int(num) > row['BEV_end'])):
                ref_nums.append(num)
            else:
                continue
        for ref_num in ref_nums:
            # store BEV number, sgRNA sequence, and primer name to get corresponding CRISPResso files 
            bev_list.append((get_bev_str(ref_num),row['sgRNA_sequence'],row['primer']))
        print(bev_list)
    
    # Call get_bev_files function which merges Alleles_frequency_tables_around_sgRNA for given sgRNA  
    merge,cols = get_bev_files(bev_list)
    
    # Calculate log-normalized reads per million for each col
    for col in ['#Reads'+col for col in cols]:
        colsum = merge[col].sum()
        merge.loc[:,str(col+';lognorm')] = merge[col].apply(lambda x: log((float(x)/float(colsum))*1000000 + 1,2))

    print(['%Reads'+col for col in cols])
    #changed to <2% for poor sequencing quality
    # Apply read count filter
    merge.loc[:,'%read_count_filter'] = merge.apply(read_count_filter,args=(['%Reads'+col for col in cols],2),axis=1) # less than 1% of all reads
    merge.loc[:,'#read_count_filter'] = merge.apply(read_count_filter,args=(['#Reads'+col for col in cols],100),axis=1) # less than 100 reads
    
    # Before translating, check if there are introns and/or UTRs in translation_ref_seq column in input file
    # introns indicated by lowercase letters in translation_ref_seq column 
    # UTRs indicated by square brackets in translation_ref_seq column 
    
    # Returns true if intron in translation_ref_seq (i.e. if lowercase letters in input translation_ref_seq)
    intron_flag = any(c.islower() for c in str(data.loc[data.index[0], 'translation_ref_seq']))
    
    # Returns true if UTR in translation_ref_seq (i.e. if square brackets in input translation_ref_seq)
    utr_flag = any(c=='[' for c in str(data.loc[data.index[0], 'translation_ref_seq']))
    
#     print(intron_flag)
    if utr_flag or intron_flag:
        filtered_input_df = pd.DataFrame()
        filtered_input_df['old_Aligned_Sequence'] = merge.loc[:,'Aligned_Sequence'].copy()
        filtered_input_df['translation_ref_seq'] = data['translation_ref_seq'].to_list()[0]
        filtered_input_df['rev_com'] = data.loc[i, 'rev_com']
    
    if utr_flag:
        print('UTRs exist')
#         filtered_input_df = pd.DataFrame()
#         filtered_input_df['Aligned_Sequence'] = merge.loc[:,'Aligned_Sequence'].copy()
#         filtered_input_df['translation_ref_seq'] = data['translation_ref_seq'].to_list()[0]
#         filtered_input_df['rev_com'] = data.loc[i, 'rev_com']
        
        # Call remove_utrs function to remove UTRs from aligned sequence before translating 
        filtered_input_df['Aligned_Sequence']= list(map(remove_utr, filtered_input_df['translation_ref_seq'], filtered_input_df['old_Aligned_Sequence'].copy(), filtered_input_df['rev_com']))
#         print(filtered_input_df)
        
#     else:
#         print('no UTRs found')


    # if introns exist, remove introns before translating 
    if intron_flag:
        print('introns exist')
#         intron_input_df = pd.DataFrame()
#         intron_input_df['Aligned_Sequence'] = merge.loc[:,'Aligned_Sequence'].copy()
#         intron_input_df['translation_ref_seq'] = data['translation_ref_seq'].to_list()[0]
#         intron_input_df['rev_com'] = data.loc[i, 'rev_com']
        if utr_flag: # if already UTR-filtered
            print('both introns and UTRs exist')
            filtered_input_df['Aligned_Sequence']= list(map(remove_introns, filtered_input_df['translation_ref_seq'], filtered_input_df['Aligned_Sequence'], filtered_input_df['rev_com']))
        else:
            print('only introns exist')
            # Call remove_introns function to remove introns from aligned sequence before translating 
            filtered_input_df['Aligned_Sequence']= list(map(remove_introns, filtered_input_df['translation_ref_seq'], filtered_input_df['old_Aligned_Sequence'], filtered_input_df['rev_com']))
    
    # if introns or UTR in translation_ref_seq
    if utr_flag or intron_flag: 
        # Call translate function to translate new (UTR-free and intron-free) sequence  
        merge.loc[:,'Translated'] = filtered_input_df['Aligned_Sequence'].apply(translate, args =(row['frame'],row['first_codon'], row['last_codon'], codon_map,))
    
    # if no introns or UTR
    else:
        print('no UTRs or introns')
        # check if Aligned Sequence needs to be reverse complemented before translating 
        rev_com = data.loc[i, 'rev_com']
        if rev_com:
            merge.loc[:,'Aligned_Sequence'] = merge.loc[:,'Aligned_Sequence'].apply(cfs.revcom)
            merge.loc[:,'Reference_Sequence'] = merge.loc[:,'Reference_Sequence'].apply(cfs.revcom)
        # Call translate function to translate Aligned Sequence (reverse complemented if necessary) 
        merge.loc[:,'Translated'] = merge.loc[:,'Aligned_Sequence'].apply(translate,args=(row['frame'],row['first_codon'], row['last_codon'], codon_map,))
    
#     print(merge['Translated'])
    return merge
'''
Called from: run
Function: joins all possible combinations of #Reads;lognorms columns from samples for given sgRNA 

INPUTS
------
row : #Reads;lognorm columns 

OUTPUTS
-------
columns joined by '_' : '#Reads_BEV_#;lognorm column_#Reads_BEV_#;lognorm column'
Ex. '#Reads_BEV_041;lognorm_#Reads_BEV_042;lognorm'  

'''

def get_corr_name(row):
    cols = [row['R1'],row['R2']]
    cols.sort()
    return '_'.join(cols)

'''
Called from: run
Function: gets combinations of samples specified in reps_for_correlation column in correlation input 

INPUTS
------
corr_input : correlation input file 
sg: guide identifier from corr_input

OUTPUTS
-------
combos : '#Reads_BEV_#;lognorm column_#Reads_BEV_#;lognorm column' for pairs specified in corr_input  

'''


def get_correlation_cols(corr_input,sg):
    corr_input = corr_input.loc[corr_input['sg'] == sg,:]
    combos = []
    for i,r in corr_input.iterrows():
        bevs = r['reps_for_correlation'].split(';')
        bevs = ['#Reads_BEV_'+get_bev_str(bev)+';lognorm' for bev in bevs]
        if len(bevs) > 1:
            combos.extend(itertools.combinations(bevs,2))
    combos = ['_'.join(combo) for combo in combos]
    return combos

hide_toggle(toggle_text_addon='process data functions')

In [13]:
'''
This function calls all previous functions to generate output tables 

INPUTS
------
input_file : input file with columns 'sg', 'translation_ref_seq', 'sgRNA_sequence', 'BEV_start', 'BEV_end', 'primer',
       'frame', 'rev_com', 'BEV_ref', 'BEV_test' described above
corr_input : input file with columns 'sg', 'reps_for_correlation' described above

OUTPUTS
-------
output files stored in output_filepath given above

'''

def run(input_file,corr_input):
    
    # List of sgRNA identifiers provided in "sg" column of input file
    sg_list = list(set(input_file['sg'].tolist())) # drop duplicates from list

    # Go through each sgRNA separately    
    for sg in sg_list:
        print(sg)
        
        # Filter input file to contain only rows for given sgRNA
        data = input_file.loc[input_file['sg'] == sg,:]
        
        # Merge all the allele read counts
        # To do this, drop the BEV_test and BEV_ref columns (which just have information needed for the LFC calculation)
        # Then drop duplicate rows
        data_dedup = data.drop_duplicates(subset=['sg','BEV_start','BEV_end','sgRNA_sequence','primer','frame','rev_com'])
        
        # Call process_data_v2 function which adds the following columns to output table:
        #read counts, lognorms, aligned sequences, reference sequences, translations 
        merge = process_data_v2(data_dedup)
        
        # Get the WT column
        merge['WT'] = merge.apply(get_wt_col,axis=1)
        
#         print(merge[merge['WT']])
        # Now, go through each row and calculate the LFC for each of the pairs specified in BEV_test and BEV_ref
        for i,r in data.iterrows():               
            merge = get_lfc_v2(r,merge)
        print('num rows in merge df:', len(merge))
        print(merge[merge['%read_count_filter'] == True])
            
        # Write out 2 files: full file (merge) and filtered file (only including alleles with > 1% reads in at least one condition)
        Path(output_filepath).mkdir(parents=True, exist_ok=True)
        merge.to_csv(output_filepath +str(r['sg'])+'_'+r['primer']+'_allele_frequency_table_around_sgRNA.csv',index=False)
        filtered = merge[merge['%read_count_filter'] == True]
        filtered.to_csv(output_filepath +str(r['sg'])+'_'+r['primer']+'_filtered_allele_frequency_table_around_sgRNA.csv',
                        index=False)            
        
        filtered_adjusted_percentage = filtered.copy()
        
        # Adjust percentages in filtered table so they total 100% 
        
        # Get #Reads columns from filtered table 
        num_reads_cols_all = [col for col in filtered_adjusted_percentage.columns if '#Reads' in col]
        # List lognorm column names that are included in #Reads cols 
        lognorm_cols = [col for col in num_reads_cols_all if 'lognorm' in col]
        # Drop lognorm columns 
        num_reads_cols = [col for col in num_reads_cols_all if col not in lognorm_cols]
        
#         print(num_reads_cols)
        
        for col in num_reads_cols:
            num_reads_col = col
            sum_reads = filtered_adjusted_percentage[num_reads_col].sum()
            print(sum_reads)
#             percentage_reads_col ='%Reads_BEV_'+ get_bev_str(num)
            percentage_reads_col = num_reads_col.replace('#', '%')
            for i in filtered_adjusted_percentage.index:
                num_reads = filtered_adjusted_percentage.loc[i, num_reads_col]
                adjusted_percentage_reads = (num_reads/sum_reads) * 100 
                filtered_adjusted_percentage.at[i, percentage_reads_col] = adjusted_percentage_reads
                
#         print(filtered_adjusted_percentage)
        
        filtered_adjusted_percentage.to_csv(output_filepath +str(r['sg'])+'_'+r['primer']+'_filtered_adj_percentage_allele_frequency_table_around_sgRNA.csv',
                index=False) 

        # Get correlations matrix of log-normalized rpm, using only alleles with > 100 reads in at least one sample
        merge = merge[merge['#read_count_filter'] == True]
        cols = [x for x in list(merge) if 'lognorm' in x]
        correlations = merge[cols].corr(method='pearson')
        correlations['R1'] = correlations.index
        correlations = correlations.melt(id_vars = 'R1', value_vars=list(correlations).remove('R1'),var_name='R2',value_name='Pearson')
        
        # Drop correlations that are not specified in corr_input
        correlations['Reps'] = correlations.apply(get_corr_name,axis=1)
        combos = get_correlation_cols(corr_input,sg)
        correlations = correlations[correlations['Reps'].isin(combos)]
        
        # Drop duplicate rows (i.e. A vs B and B vs A)
        correlations = correlations.drop_duplicates(subset=['Reps'])
        
        # Write to file
        Path(output_filepath + "corr_outputs/#read_count_filter_pearson/").mkdir(parents=True, exist_ok=True)
        correlations.to_csv(output_filepath + "corr_outputs/#read_count_filter_pearson/sg" +str(r['sg'])+'_'+r['primer']+'_correlations.csv')
#         break
    return 

hide_toggle(toggle_text_addon='run function')    

In [14]:
import math
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable

rcParams['font.family'] = 'monospace' #so translated sequences are aligned
rcParams['font.monospace'] = 'Arial'
rcParams['font.size']: 12.0

'''
This function generates allele-level heatmaps using log-fold changes for each validation condtion
annotated with the corresponding allele amino acid sequences

INPUTS
------
df : input file with columns 'sg', 'translation_ref_seq', 'sgRNA_sequence', 'BEV_start', 'BEV_end', 'primer',
       'frame', 'rev_com', 'BEV_ref', 'BEV_test' described above
vmin : sets minimum value for heatmap color bar
vmax: sets maximum value for heatmap color bar
filepath: same as output_filepath specified above where allele frequency output tables are saved 
time1: (optional) string describing early time point for table header, e.g. 'Day7' (default: 'Time1')
time2: (optional) string describing late time point for table header, e.g. 'Day14' (default: 'Time2')

OUTPUTS
-------
1. allele-level heatmpas with corresponding allele amino acid sequences annotated
2. tables with average % reads for each allele at early and late time points 

'''

def heatmaps(df,vmin,vmax,filepath, time1 = 'Time1', time2 = 'Time2', timepoint_df=None, filename = 'translation_heatmap', **kwargs):

    num_rows = len(df)
    num_cols = 2
    fig,axs = plt.subplots(num_rows, num_cols, figsize=(num_cols*4,num_rows*4))
    plt.subplots_adjust(wspace=0.5, hspace = 0.5)
    
    row_count = 0
    
    for num, (i,r) in enumerate(df.iterrows()):

        sg = r['sg']
#         print('before plot sg: ', sg)
        primer = r['primer']
#         location = filepath+str(sg)+'_'+primer+'_filtered_allele_frequency_table_around_sgRNA.csv'
        location = filepath+str(sg)+'_'+primer+'_filtered_adj_percentage_allele_frequency_table_around_sgRNA.csv'

        to_plot = pd.read_csv(location)

        avgLFC_col = [col for col in to_plot.columns if 'AvgLFC' in col][0]

        to_plot.sort_values(by=avgLFC_col,ascending=False,inplace=True)
        
        # Earlier time point -> Time1
        time1_avg_col = 'Avg%Reads_' + time1
        
        # Later time point -> Time 2
        time2_avg_col = 'Avg%Reads_' + time2
        
        if timepoint_df is None: # Default to BEV_ref as Time1 samples and BEV_test as Time2 samples
            
            # Default Time 1 (Earlier time point) samples  = BEV_ref columns 
            time1_samples = r['BEV_ref'].split(';') # convert to list of BEV numbers

            # Default Time 2 (Later time point) samples  = BEV_ref columns 
            time2_samples = r['BEV_test'].split(';') # convert to list of BEV numbers 
            
        else: # if timepoint_df provided
            # Get Time 1 columns from df based on time1 parameter 
            sg_timepoint_df = timepoint_df[timepoint_df['sg']==sg]
            time1_sample_df = sg_timepoint_df[sg_timepoint_df['time_point']==time1].reset_index(drop=True)
            time1_samples = time1_sample_df.loc[0, 'BEV_nums'].split(';')
            
            # Get Time 2 columns from df based on time2 parameter 
            sg_timepoint_df = timepoint_df[timepoint_df['sg']==sg]
            time2_sample_df = sg_timepoint_df[sg_timepoint_df['time_point']==time2].reset_index(drop=True)
            time2_samples = time2_sample_df.loc[0, 'BEV_nums'].split(';')
        
        # Lists to store correctly formatted %Reads column names 
        time1_cols = [] 
        time2_cols = []
        
        # %Reads columns named "%Reads_BEV_" + get_bev_str(int(BEV_num))
        for num in time1_samples:
            col_name = "%Reads_BEV_" + get_bev_str(int(num))
            time1_cols.append(col_name)  

        to_plot[time1_avg_col] = round(to_plot[time1_cols].apply(np.mean,axis=1),1).apply(str)#+'%'

        # %Reads columns named "%Reads_BEV_" + get_bev_str(int(BEV_num))
        for num in time2_samples:
            col_name = "%Reads_BEV_" + get_bev_str(int(num))
            time2_cols.append(col_name)  

        to_plot[time2_avg_col] = round(to_plot[time2_cols].apply(np.mean,axis=1),1).apply(str)#+'%'

        to_plot['Label'] = to_plot['Translated']

        reads_df = to_plot[[time1_avg_col, time2_avg_col]]

        to_plot.loc[to_plot['WT'],'Label'] = to_plot.loc[to_plot['WT'],'Label'].values[0] + '<wt' 

        heatmap_df = to_plot[['Label',[col for col in to_plot.columns if 'AvgLFC' in col][0]]].reset_index(drop = True)
        
        if row_count < num_rows:
            # HEATMAP
            if num_rows == 1: 
                heatmap_ax = axs[0]
            else: 
                heatmap_ax = axs[row_count, 0]
            
            cmap = LinearSegmentedColormap.from_list(name='test', colors=['#8DA0CB','white','#F8774F'])
            sns.heatmap(heatmap_df.set_index('Label'),cmap=cmap,vmin=vmin,vmax=vmax,ax=heatmap_ax,annot=True, yticklabels = True, fmt='.1f')
            label_list = [item for item in heatmap_ax.get_yticklabels()]
            for label in label_list:
                if '<wt' in label.get_text():
                    label.set_color('red')
            plt.setp(heatmap_ax.get_yticklabels(), rotation=0, ha="left", size = 12)#,rotation_mode="anchor")

            yax = heatmap_ax.get_yaxis()
            # find the maximum width of the label on the major ticks
            pad = max(T.label.get_window_extent().width for T in yax.majorTicks)
            yax.set_tick_params(pad=pad)
            heatmap_ax.set_ylabel('', rotation = 0)
            heatmap_ax.set_title(sg)
                    
            bbox = heatmap_ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
            width, height = bbox.width, bbox.height

            # TABLE
            cell_text = []
            for row in range(len(reads_df)):
                cell_text.append(reads_df.iloc[row])
            
            if num_rows == 1: 
                table_ax = axs[1]
            else: 
                table_ax = axs[row_count, 1]
            #print('table position:', table_ax)
            translations = list(heatmap_df['Label'])
            #print(translations)
            cell_height = height/(2*(len(translations)))

            reads_table = table_ax.table(cellText=cell_text, colLabels=reads_df.columns, loc='center', cellLoc = 'center', edges = 'open')#, bbox = [0,0, width, height/2.3])
            reads_table.auto_set_font_size(False)
            reads_table.set_fontsize(12)
            reads_table.auto_set_column_width(col=list(range(len(reads_df.columns))))
            #reads_table.scale(1, 1.5)
            cellDict = reads_table.get_celld()
            
            for i in range(0,len(reads_df.columns)):
                cellDict[(0,i)].set_height(cell_height)
                for j in range(1,len(reads_df)+1):
                    cellDict[(j,i)].set_height(cell_height)
            table_ax.set_axis_off()
            

            row_count+=1
#             print(row_count)
#             break
    # Create path to Figures folder if doesn't exist already
    Path(filepath + '/Figures/').mkdir(parents=True, exist_ok=True)
    full_filepath = filepath + 'Figures/' + filename +'.pdf'
    print(full_filepath)
    plt.savefig(full_filepath,bbox_inches="tight")

hide_toggle(toggle_text_addon='heatmap function')

## User inputs

<font color='blue'> Please follow steps indicated in blue, then run the notebook to generate output files. If the files are formatted as described in the documentation, the code in the 'Functions' section should not need to be altered. </font> 

### Load input files

**Metainformation file** 

<font color='blue'> <b>Step 1:</b> Create metainformation input file in the following format. </font>

For sequence inputs please refer to the [Sequence_Orientation_Documentation](../docs/Sequence_Orientation_Documentation.html).

**Columns**: 

* **sg** : sg identifier 
* **sgRNA_sequence** : sequence of sgRNA as designed 
* **translation_ref_seq**: reference sequence outputted by CRISPResso formatted such that any intronic sequences are lower-case, exons are upper-case, and UTRs are indicated by square brackets (if applicable) <u> must be sequence on strand that is being translated; may not necessarily be the same strand as the sgRNA sequence</u> 
    * Ex. <font color='grey'>tgtcttttctatgatctctttag</font><font color='green'>GGGTGACCCAGTCTATT</font>
* **BEV_start** : BEV number for first sample in sg
* **BEV_end** : BEV number for last sample in sg
* **primer** : name of primer pair (joined by '\_') used to amplify genomic locus as mentioned in sample name
    * Ex. <font color='purple'>F_C12</font><font color = 'blue'><b>_</b></font><font color='green'>R_C12</font>
* **frame** : frame for translation (manually determined for each sg / primer pair); position of first coding nucleotide in reference sequence within codon; frame can be 1, 2, 3
    * Ex. given reference sequence: tgtcttttctatgatctctttag<font color='green'>**G**</font>G|GTG|ACC|CAG|TCT|ATT 
        since the first coding nucleotide of the reference sequence (<font color='green'><b>G</b></font>) is the 2nd nucleotide in its codon 
        (\_<font color='green'><b>G</b></font>G) &rightarrow; frame = 2
* **first_codon** : first codon for translation 
* **last_codon** : last codon for translation 
* **rev_com** : samples for which reference sequence is on reverse strand 
* **BEV_ref** : reference sample(s) for log-fold change (LFC) calculation (i.e. early time point, empty vector, etc.); if multiple BEV numbers are given, they should be separated by ';', and they will be treated as replicates that will be averaged
* **BEV_test** : test sample(s) for LFC calculation; if multiple BEV numbers are given, they should be separated by ';', and they will be treated as replicates that will be averaged

**Example input:**


| sg      | sgRNA_sequence       | translation_ref_seq                                  | BEV_start | BEV_end | primer        | frame | first_codon| last_codon| rev_com | BEV_ref | BEV_test |
| ------- | -------------------- | ---------------------------------------- |  -------: |  -----: | ------------- |  ----|----|---: | ------: | ------- | -------- |
| 397   | GTCACCCCTAAAGAGATCAT | tgtcttttctatgatctctttagGGGTGACCCAGTCTATT | 7         | 12      |F_C12_R_C12 |  2    |TGG|ATT| True    | 5;6     | 9;10     |






<font color='blue'> <b> Step 2: </b> Enter filepath to metainformation input file here </font> 

In [ ]:
# input_file = pd.read_csv('MOLM13_BCL2_allvalidation/Metainfo_input_BCL2_CBE_first_last_codon.csv')
input_filepath = input("Please enter input filepath here: ")
input_file = pd.read_csv(input_filepath)
input_file.head()

In [ ]:
#check for NaN values i.e. blank rows
if input_file.isnull().values.any(): 
    input_file = clean_input_file(input_file)

In [ ]:
check_input_file(input_file)

**Correlation Metainformation file** 

<font color='blue'><b>Step 3:</b> Create correlation input file in the following format  </font> 

**Columns**: 

* **sg** : sg identifier 
* **reps_for_correlation** : semicolon-separated BEV numbers of which to calculate the pairwise Pearson correlation of the log-normalized read counts

**Example input:**
    
| sg      | reps_for_correlation |
| ------- | -------------------: | 
| 397     | 7;8 | 
| 397     | 9;10 | 
| 397     | 11;12 | 


<font color='blue'><b>Step 4:</b> Enter filepath to correlation input file here  </font> 

In [ ]:
# corr_input = pd.read_csv('MOLM13_BCL2_allvalidation/Correlation_input_CBE.csv')
corr_corr_inputpath = input("Please enter correlation input filepath here: ")
corr_input = pd.read_csv(corr_corr_inputpath)
corr_input.head()

### Specify folder filepaths 

<font color='blue'><b>Step 5:</b> Enter filepath to folder containing CRISPResso output files here. Please make sure that the filepath does not begin with a '/' but does end in a '/'.  </font> 

Please note that each folder containing CRISPResso output files for individual samples within the given folder should be named in the format 'CRISPResso_on_'+bev+'\_'+
primer, where bev = ('BEV' or 'NGBEV') + sample_number and primer = primer name. 
Ex. <font color='grey'>CRISPResso_on</font><font color='purple'>_BEV_001</font><font color='green'>_F2_R2</font>

In [ ]:
global bev_string_id
bev_string_id = input('Please enter either \'BEV\' or \'NGBEV\' to indicate which string is used when naming your CRISPResso files.')
if ((bev_string_id != 'BEV') and (bev_string_id != 'NGBEV')):
    raise Exception('Invalid input. Please enter either \'BEV\' or \'NGBEV\' to specify which string is used in CRISPResso file names. Be careful not to add any extra spaces.')



In [ ]:
global CRISPResso_filepath 
# CRISPResso_filepath = 'MOLM13_BCL2_allvalidation/CRISPRessoBatch_on_FR_batch_file/'
CRISPResso_filepath = input("Please enter CRISPResso filepath here: ")

<font color='blue'><b>Step 6:</b> Enter filepath to folder where the files generated by this notebook will be stored. Please make sure that the filepath does not begin with a '/' but does end in a '/'. If the folders in this file path do not currently exist, they will be created when the notebook is run.  </font> 

In [ ]:
# Filepath to store allele_freq output tables 
global output_filepath 
# output_filepath = 'MOLM13_BCL2_allvalidation/Results/CBE_v2/2percent_filter/'
output_filepath = input("Please enter output folder filepath here: ")

### Run functions

Now, we run the two input files and produce allele tables for all sgRNAs.

In [ ]:
run(input_file,corr_input)

## Heatmap

Finally, we generate allele-level heat maps annotated with corresponding allele amino acid sequences and tables with average % reads for each allele at early and late time points.

By default, early time point samples are considered to be the same as the BEV_ref samples and the late time point samples are considered to be the BEV_test samples. If this is not the case (e.g.,  BEV_test samples span multiple time points), please upload a file with the following columns: 

**Columns**: 

* **sg** : sg identifier 
* **Time point** : string that identifies time point 
* **BEV_num** : semicolon-separated BEV numbers corresponding to samples at that time point 

**Example input:**
    
| sg      |time_point | BEV_nums |
| ------- |------- | -------------------: | 
| 397     |D8      |  7;8 | 
| 397     |D14      | 9;10 | 
| 397     |D21      | 11;12 | 


In [ ]:
global timepoint_input_file

# Check if default settings work for user
default_y_or_n = input("Do the default time point sample assignments work for you? Please enter 'y' or 'n'. If 'n', you will be asked to enter the path to a time point input file as described above. ")

In [ ]:
default_y_or_n = default_y_or_n.lower()
if default_y_or_n == 'y':
    timepoint_input_file = None
#     print('y')
elif default_y_or_n == 'n':
    timepoint_input_filepath = input("Please enter input filepath here: ")
    # timepoint_input_file = pd.read_csv('AnnabelData/timepoint_df_test.csv')
    timepoint_input_file = pd.read_csv(timepoint_input_filepath)
#     print('n')
else:
    raise Exception('Invalid input. Please enter either \'y\' or \'n\' and re-run the cell.')

In [ ]:
heatmaps(df = input_file,
         vmin = -2,
         vmax = 2,
         filepath = output_filepath,
         time1 = 'D7',
         time2 = 'D21',
         timepoint_df = timepoint_input_file,
         filename = 'BCL2_CBE_heatmap'
        )